## Ipython Notebooks For Triple-Bar

Add the package path for testing

In [1]:
import sys
sys.path.append('E:\\git_folder\\quant_models')

In [2]:
import pandas as pd
from util.utils import get_daily_vol, sample_df, filter_df_time
data_fn = "././data/xu1_1min.csv"
df_price = pd.read_csv(data_fn)
test_df = df_price.copy()
test_df.loc[:,'datetime'] = pd.to_datetime(test_df.datetime)
test_df.set_index('datetime', inplace=True)
df_testprice = test_df.loc[test_df.index > pd.to_datetime('2019-11-01')]
df_sampled = sample_df(df_testprice, '30T')
print(df_sampled.count())
df_sampled = df_sampled.reindex(df_sampled.index.intersection(df_testprice.index))
print(df_sampled.count())

open      1402
close     1402
low       1402
high      1402
volume    1402
dtype: int64
open      1219
close     1219
low       1219
high      1219
volume    1219
dtype: int64


In [3]:
df_sampled.head()

,open,close,low,high,volume
datetime,,,,,
2019-11-01 00:30:00,13865.0,13857.5,13850.0,13865.0,686.0
2019-11-01 01:00:00,13855.0,13855.0,13850.0,13860.0,438.0
2019-11-01 01:30:00,13855.0,13850.0,13850.0,13862.5,584.0
2019-11-01 02:00:00,13850.0,13852.5,13847.5,13857.5,288.0
2019-11-01 02:30:00,13852.5,13845.0,13840.0,13852.5,428.0


## Start Testing

In [4]:
df_sampled.tail(2)
from labeling import get_events

In [49]:
timedelta = pd.Timedelta('5 hours')
nearest_index = df_sampled.index.searchsorted(df_sampled.index + timedelta)

In [50]:
nearest_index

array([   9,    9,    9, ..., 1219, 1219, 1219], dtype=int64)

In [51]:
from labeling import add_vertical_barrier

In [52]:
vertical_barrier_times = add_vertical_barrier(df_testprice.open, df_sampled.index, num_hours=5)

In [53]:
vertical_barrier_times.tail(20)

datetime
2019-12-17 18:30:00   2019-12-17 23:30:00
2019-12-17 19:00:00   2019-12-18 00:00:00
2019-12-17 20:00:00   2019-12-18 01:00:00
2019-12-17 20:30:00   2019-12-18 01:33:00
2019-12-17 21:00:00   2019-12-18 02:00:00
2019-12-17 21:30:00   2019-12-18 02:30:00
2019-12-17 22:00:00   2019-12-18 03:00:00
2019-12-17 22:30:00   2019-12-18 03:30:00
2019-12-17 23:00:00   2019-12-18 04:00:00
2019-12-17 23:30:00   2019-12-18 04:32:00
2019-12-18 00:00:00   2019-12-18 05:00:00
2019-12-18 00:30:00   2019-12-18 08:59:00
2019-12-18 01:00:00   2019-12-18 08:59:00
2019-12-18 02:00:00   2019-12-18 08:59:00
2019-12-18 02:30:00   2019-12-18 08:59:00
2019-12-18 03:00:00   2019-12-18 08:59:00
2019-12-18 03:30:00   2019-12-18 08:59:00
2019-12-18 04:00:00   2019-12-18 09:00:00
2019-12-18 05:00:00   2019-12-18 10:00:00
2019-12-18 09:00:00   2019-12-18 14:00:00
Name: datetime, dtype: datetime64[ns]

In [54]:
target_vert = get_daily_vol(df_sampled)

In [55]:
target = target_vert['vol']
#t_events = df_sampled.index

In [56]:
#target = target.loc[t_events]

In [57]:
#target_vert.head()

In [58]:
# = target.reindex(t_events)

In [59]:
#targetv1.dropna(inplace=True)

In [60]:
#target = targetv1.copy()

In [61]:
events = get_events(df_testprice.open, target, df_sampled.index, min_ret=0.001, 
          num_threads = 3, vertical_barrier_times=vertical_barrier_times)

2019-12-23 17:23:27.913754 100.0% apply_pt_sl_on_ent done after 0.07 minutes. Remaining 0.0 minutes.


In [92]:
hh.head()

,ret,trgt,bin
datetime,,,
2019-11-02 01:30:00,0.000879,0.000769,1
2019-11-02 02:00:00,-0.000527,0.001090,0
2019-11-02 02:30:00,-0.001404,0.000960,-1
2019-11-02 03:00:00,0.001054,0.000844,1
2019-11-02 03:30:00,0.001230,0.000901,1


In [62]:
final_hh = get_bins(events, df_testprice.open)

In [63]:
final_hh.head()

In [64]:
events = pd.concat({'ent': vertical_barrier_times, 'trgt': target, 'side': side_}, axis=1)

In [65]:
events = events.dropna(subset=['side'])

In [66]:
events = events.fillna(value={'trgt': 0.001})

In [67]:
events = events.dropna(subset=['trgt'])

In [68]:
events.head()


,ent,side,trgt
datetime,,,
2019-11-02 01:30:00,2019-11-04 08:59:00,1.0,0.000769
2019-11-02 02:00:00,2019-11-04 08:59:00,1.0,0.001090
2019-11-02 02:30:00,2019-11-04 08:59:00,1.0,0.000960
2019-11-02 03:00:00,2019-11-04 08:59:00,1.0,0.000844
2019-11-02 03:30:00,2019-11-04 08:59:00,1.0,0.000901


In [69]:
# Snippet 3.1, page 44, Daily Volatility Estimates
from util.multiprocess import mp_pandas_obj
from labeling import apply_pt_sl_on_ent

In [70]:
num_threads = 2

In [71]:
events.head(2000)

,ent,side,trgt
datetime,,,
2019-11-02 01:30:00,2019-11-04 08:59:00,1.0,0.000769
2019-11-02 02:00:00,2019-11-04 08:59:00,1.0,0.001090
2019-11-02 02:30:00,2019-11-04 08:59:00,1.0,0.000960
2019-11-02 03:00:00,2019-11-04 08:59:00,1.0,0.000844
2019-11-02 03:30:00,2019-11-04 08:59:00,1.0,0.000901
2019-11-02 04:00:00,2019-11-04 08:59:00,1.0,0.000949
2019-11-02 04:30:00,2019-11-04 08:59:00,1.0,0.000894
2019-11-04 09:00:00,2019-11-04 14:00:00,1.0,0.009570
2019-11-04 09:30:00,2019-11-04 14:30:00,1.0,0.011794


In [72]:
first_touch_dates = apply_pt_sl_on_ent(df_testprice.open, events, pt_sl_, events.index)

In [73]:
first_touch_dates.head(5)

,ent,sl,pt
datetime,,,
2019-11-02 01:30:00,2019-11-04 08:59:00,NaT,2019-11-02 01:51:00
2019-11-02 02:00:00,2019-11-04 08:59:00,2019-11-02 03:12:00,NaT
2019-11-02 02:30:00,2019-11-04 08:59:00,2019-11-02 02:36:00,NaT
2019-11-02 03:00:00,2019-11-04 08:59:00,NaT,2019-11-02 03:38:00
2019-11-02 03:30:00,2019-11-04 08:59:00,NaT,2019-11-02 04:09:00


In [74]:
events.head(5)

,ent,side,trgt
datetime,,,
2019-11-02 01:30:00,2019-11-04 08:59:00,1.0,0.000769
2019-11-02 02:00:00,2019-11-04 08:59:00,1.0,0.001090
2019-11-02 02:30:00,2019-11-04 08:59:00,1.0,0.000960
2019-11-02 03:00:00,2019-11-04 08:59:00,1.0,0.000844
2019-11-02 03:30:00,2019-11-04 08:59:00,1.0,0.000901


In [75]:
gg = first_touch_dates.dropna(how='all').min(axis=1)

In [76]:
gg.head()

datetime
2019-11-02 01:30:00   2019-11-02 01:51:00
2019-11-02 02:00:00   2019-11-02 03:12:00
2019-11-02 02:30:00   2019-11-02 02:36:00
2019-11-02 03:00:00   2019-11-02 03:38:00
2019-11-02 03:30:00   2019-11-02 04:09:00
dtype: datetime64[ns]

In [77]:
events['ent'] = first_touch_dates.dropna(how='all').min(axis=1)

In [78]:
events.head()

,ent,side,trgt
datetime,,,
2019-11-02 01:30:00,2019-11-02 01:51:00,1.0,0.000769
2019-11-02 02:00:00,2019-11-02 03:12:00,1.0,0.001090
2019-11-02 02:30:00,2019-11-02 02:36:00,1.0,0.000960
2019-11-02 03:00:00,2019-11-02 03:38:00,1.0,0.000844
2019-11-02 03:30:00,2019-11-02 04:09:00,1.0,0.000901


In [79]:
#events = events.drop('side', axis=1)
events['pt'] = 1
events['sl'] = 1

In [80]:
events.head(2)

,ent,side,trgt,pt,sl
datetime,,,,,
2019-11-02 01:30:00,2019-11-02 01:51:00,1.0,0.000769,1,1
2019-11-02 02:00:00,2019-11-02 03:12:00,1.0,0.001090,1,1


In [81]:

first_touch_dates = mp_pandas_obj(func=apply_pt_sl_on_ent,
                                  pd_obj=('molecule', events.index),
                                  num_threads=2,
                                  df_price=df_testprice.open,
                                  events=events,
                                  pt_sl=pt_sl_)


2019-12-23 17:23:36.401057 100.0% apply_pt_sl_on_ent done after 0.07 minutes. Remaining 0.0 minutes.


In [82]:
#hh = apply_pt_sl_on_ent(df_testprice, events, pt_sl_, events.index)

In [83]:
events = events.drop('side', axis=1)

In [84]:
events['pt'] = 1
events['sl'] = 1

In [85]:
from labeling import get_bins, drop_labels

In [86]:
events.head(50)

,ent,trgt,pt,sl
datetime,,,,
2019-11-02 01:30:00,2019-11-02 01:51:00,0.000769,1,1
2019-11-02 02:00:00,2019-11-02 03:12:00,0.001090,1,1
2019-11-02 02:30:00,2019-11-02 02:36:00,0.000960,1,1
2019-11-02 03:00:00,2019-11-02 03:38:00,0.000844,1,1
2019-11-02 03:30:00,2019-11-02 04:09:00,0.000901,1,1
2019-11-02 04:00:00,2019-11-04 08:59:00,0.000949,1,1
2019-11-02 04:30:00,2019-11-04 08:59:00,0.000894,1,1
2019-11-04 09:00:00,2019-11-04 14:00:00,0.009570,1,1
2019-11-04 09:30:00,2019-11-04 14:30:00,0.011794,1,1


In [87]:
hh = get_bins(events, df_sampled.open)

In [88]:
hh.head(100)

,ret,trgt,bin
datetime,,,
2019-11-02 01:30:00,0.000879,0.000769,1
2019-11-02 02:00:00,-0.000527,0.001090,0
2019-11-02 02:30:00,-0.001404,0.000960,-1
2019-11-02 03:00:00,0.001054,0.000844,1
2019-11-02 03:30:00,0.001230,0.000901,1
2019-11-02 04:00:00,-0.000351,0.000949,0
2019-11-02 04:30:00,-0.000702,0.000894,0
2019-11-04 09:00:00,0.002634,0.009570,0
2019-11-04 09:30:00,0.002984,0.011794,0


In [89]:
new_events = drop_labels(hh)

In [90]:
new_events.bin.min()

-1

In [91]:
hh.bin.value_counts()

 0    1032
 1      78
-1      59
Name: bin, dtype: int64